<font color = green >

## Home Task 

</font>


<font color = green >

### Load data 

</font>

[Sentiment Analysis Dataset](https://www.kaggle.com/sonaam1234/sentimentdata)

alternative source: 
<br>
[rt-polaritydata](https://github.com/dennybritz/cnn-text-classification-tf/tree/master/data/rt-polaritydata)

alternative source: 
<br>
[Movie Review Data](http://www.cs.cornell.edu/people/pabo/movie-review-data)

Each line in these two files corresponds to a single snippet (usually containing roughly one single sentence); all snippets are down-cased.  
[More info about dataset](https://www.cs.cornell.edu/people/pabo/movie-review-data/rt-polaritydata.README.1.0.txt)



In [1]:
fn='rt-polarity.neg'
with open(fn, "r",encoding='utf-8', errors='ignore') as f: # some invalid symbols encountered 
    content = f.read()  
texts_neg=  content.splitlines()
print ('len of texts_neg = {:,}'.format (len(texts_neg)))
for review in texts_neg[:5]:
    print ( '\n', review)

len of texts_neg = 5,331

 simplistic , silly and tedious . 

 it's so laddish and juvenile , only teenage boys could possibly find it funny . 

 exploitative and largely devoid of the depth or sophistication that would make watching such a graphic treatment of the crimes bearable . 

 [garbus] discards the potential for pathological study , exhuming instead , the skewed melodrama of the circumstantial situation . 

 a visually flashy but narratively opaque and emotionally vapid exercise in style and mystification . 


In [2]:
fn='rt-polarity.pos'

with open(fn, "r",encoding='utf-8', errors='ignore') as f:
    content = f.read()
texts_pos=  content.splitlines()
print ('len of texts_pos = {:,}'.format (len(texts_pos)))
for review in texts_pos[:5]:
    print ('\n', review)

len of texts_pos = 5,331

 the rock is destined to be the 21st century's new " conan " and that he's going to make a splash even greater than arnold schwarzenegger , jean-claud van damme or steven segal . 

 the gorgeously elaborate continuation of " the lord of the rings " trilogy is so huge that a column of words cannot adequately describe co-writer/director peter jackson's expanded vision of j . r . r . tolkien's middle-earth . 

 effective but too-tepid biopic

 if you sometimes like to go to the movies to have fun , wasabi is a good place to start . 

 emerges as something rare , an issue movie that's so honest and keenly observed that it doesn't feel like one . 


In [3]:
import pandas as pd
import numpy as np
import nltk
from nltk.corpus import movie_reviews 
from nltk.corpus import stopwords
import nltk
from nltk.tokenize import RegexpTokenizer
import random

In [4]:
t = texts_pos+texts_neg
t[0]
te = ''.join(t)

In [5]:
def preprocess(text): 
    tokenizer = RegexpTokenizer(r'\w+') 
    return tokenizer.tokenize(text.lower())

all_words = preprocess(te)


In [6]:
all_words=nltk.FreqDist(all_words)
most_common_words = list(zip(*all_words.most_common()))[0]


In [7]:
def remove_stop_words(words):
    stop_words = set(stopwords.words('english'))  
    return [w for w in words if w not in stop_words]
most_common_words_filtered = remove_stop_words(most_common_words)

In [10]:
word_features = most_common_words_filtered [:3000]
documents = [(list(movie_reviews.words(file_id)), category) 
             for category in movie_reviews.categories() 
             for file_id in movie_reviews.fileids(category)]

random.shuffle(documents) 
documents= documents[:500]

def find_features(review_tokens):
    return {w: w in set(review_tokens) for w in word_features} 

data_set= [(find_features(review_tokens), category) for (review_tokens, category) in documents]
split_on = int(len(data_set)*.8)
X_y_train= data_set[:split_on]
X_y_test = data_set[split_on:]

clf= nltk.NaiveBayesClassifier.train(X_y_train) 


print(nltk.classify.accuracy(clf, X_y_test)*100)

clf.show_most_informative_features(15)

85.0
Most Informative Features
             masterpiece = True              pos : neg    =      9.0 : 1.0
                 unfunny = True              neg : pos    =      8.9 : 1.0
                   blade = True              pos : neg    =      8.3 : 1.0
              remembered = True              pos : neg    =      7.6 : 1.0
               laughable = True              neg : pos    =      7.1 : 1.0
            conventional = True              pos : neg    =      6.9 : 1.0
                 decades = True              pos : neg    =      6.9 : 1.0
                    dull = True              neg : pos    =      6.8 : 1.0
                    rare = True              pos : neg    =      6.7 : 1.0
                    lame = True              neg : pos    =      6.5 : 1.0
                    adam = True              neg : pos    =      6.5 : 1.0
                   grade = True              neg : pos    =      6.5 : 1.0
             outstanding = True              pos : neg    =      6.3 

In [11]:
col = ['Reviews', 'Rating_binary']
df = pd.DataFrame(columns=col)

def rew(rew,rat):

    global col
    global df

    list_col = [rew, rat]

    F = pd.DataFrame([list_col], columns=col)
    df = df.append(F)

for r in texts_pos:
    rew(r, 1)

for r in texts_neg:
    rew(r, 0)


In [12]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(df['Reviews'],df['Rating_binary'],random_state=0)

In [13]:
from sklearn.feature_extraction.text import CountVectorizer 
vect = CountVectorizer().fit(X_train) 


In [14]:
X_train_vectorized = vect.transform(X_train) 


In [15]:
df = pd.DataFrame(X_train_vectorized[0].toarray(), index= ['value']).T


In [16]:
[vect.get_feature_names()[index] for index in df[df['value']>0].index.values]

['about',
 'as',
 'been',
 'cutting',
 'fresh',
 'have',
 'hollywood',
 'instead',
 'is',
 'issue',
 'last',
 'of',
 'satire',
 'should',
 'variety',
 'week',
 'what']

In [17]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score
from sklearn.metrics import f1_score

In [18]:
y_train1=y_train.astype('int')
y_test1=y_test.astype('int')

In [19]:
clf = LogisticRegression(max_iter=2000).fit(X_train_vectorized, y_train1) 

In [20]:
predictions = clf.predict(vect.transform(X_test)) 
print('f1: ', f1_score(y_test1, predictions)) 
scores = clf.decision_function(vect.transform(X_test)) 
print('AUC: ', roc_auc_score(y_test1, scores)) 

f1:  0.7782805429864252
AUC:  0.8472062608495887


In [21]:
feature_names = np.array(vect.get_feature_names())
sorted_coef_index = clf.coef_[0].argsort()
clf.coef_.shape, clf.coef_[0].shape, sorted(clf.coef_[0])[:10], sorted(clf.coef_[0])[-11:-1], 

((1, 16021),
 (16021,),
 [-2.287303923226548,
  -1.834743207989059,
  -1.8051071449693172,
  -1.660050440471349,
  -1.6233773186487392,
  -1.619425827528814,
  -1.5768822688401176,
  -1.5561489186290967,
  -1.5224796011119384,
  -1.5174272775761568],
 [1.3894967026182408,
  1.4084225640363601,
  1.4219326279639313,
  1.4612554578947061,
  1.5041495932498896,
  1.5069489085409427,
  1.532535263757047,
  1.6376105361852695,
  1.6412227149458103,
  1.6604110815247335])

In [22]:
print('Smallest coefs:\n{}\n'.format(feature_names[sorted_coef_index[:10]]))
print('Largest Coefs: \n{}'.format(feature_names[sorted_coef_index[:-11:-1]]))


Smallest coefs:
['dull' 'boring' 'waste' 'problem' 'worst' 'neither' 'bore' 'tv' 'suffers'
 'supposed']

Largest Coefs: 
['masterpiece' 'thanks' 'enjoyable' 'entertaining' 'powerful' 'remarkable'
 'solid' 'engrossing' 'provides' 'skin']


<font color = green >

## Learn more
</font>

sklearn.feature_extraction.text.CountVectorizer
<br>
https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html

Bag-of-words model
<br>
https://en.wikipedia.org/wiki/Bag-of-words_model

tf–idf
<br>
https://en.wikipedia.org/wiki/Tf%E2%80%93idf

sklearn.feature_extraction.text.TfidfVectorizer
<br>
https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html

Applied Text Mining in Python
<br>
https://www.coursera.org/learn/python-text-mining/home/welcome

Natural Language Processing tutorial
<br>
https://pythonprogramming.net/tokenizing-words-sentences-nltk-tutorial/